In [48]:
import numpy as np
import pandas as pd
from models import TSP, HopfieldNetwork

In [49]:
def read_instance(filepath):
    with open(f"{filepath}", "r") as f:
        content = f.readlines()
        content[0] = content[0].split(":")[-1].strip()[1:-1]
        tour = np.array([int(x) for x in content[0].split(",")])
        cost = float(content[1].split(":")[-1].strip())
        return (tour, cost)

In [56]:
instance = read_instance("test/tsp/instance2.sol")
print(instance[-1])

1039.77


In [57]:
tsp = TSP.from_instance_file("test/tsp/instance2.tsp", A=50000, B=50000, C=2000, D=1)

In [58]:
import gurobipy as gp
from gurobipy import Model, GRB, QuadExpr
options = {}
with gp.Env(params=options) as env, gp.Model(env=env) as model:
    # Formulate problem
    problem = tsp
    # Define the interaction matrix (Q) and field vector (c)
    Q = problem.T[-2] # Interaction matrix
    c = problem.I # Field vector
    
    # Create a new Gurobi model
    model = Model("Qua")
    
    # Add binary variables
    n = Q.shape[0]
    var = [model.addVar(vtype=GRB.BINARY, name=f"x{i}") for i in range(n)]
    
    # Set up the quadratic objective
    objective = QuadExpr()
    
    # Add quadratic terms from Q
    for i in range(n):
        for j in range(n):
            if Q[i, j] != 0:
                objective.add(-0.5*Q[i, j] * var[i] * var[j])
    
    # Add linear terms from c
    for i in range(n):
        if c[i] != 0:
            objective.add(-c[i] * var[i])
    
            
    objective.addConstant(problem.offset)
    
    # Set the objective
    model.setObjective(objective, GRB.MINIMIZE)
    model.setParam("MIPFocus", 1)
    
    # Optimize the model
    model.optimize()
    
    
    gurobi_solution = []
    # Display results
    if model.status == GRB.OPTIMAL:
        print("Optimal value:", model.objVal)
        for v in var:
            print(f"{v.varName} = {v.x}")
            gurobi_solution.append(v.x)
    else:
        print("No optimal solution found.")
        for v in var:
            print(f"{v.varName} = {v.x}")
            gurobi_solution.append(v.x)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2595968
Academic license 2595968 - for non-commercial use only - registered to s0___@uni-bonn.de


KeyboardInterrupt: 

In [53]:
from models import HopfieldNetwork
hopfield = HopfieldNetwork(tsp.T, tsp.I, tsp.offset)

print(len(tsp.interpret(np.array(gurobi_solution))))
tsp_sol = tsp.interpret(np.array(gurobi_solution))
print(tsp.interpret(np.array(gurobi_solution)))
print(tsp.get_cost(np.array(gurobi_solution)))
def is_legal(arr):
    print(np.unique(arr))
    return len(np.unique(arr)) == len(arr)

print(is_legal(tsp_sol))
print(np.array(gurobi_solution).reshape((tsp.n, tsp.n)))
print(hopfield.compute_energy(np.array(gurobi_solution), debug=True))

25
[23.  4. 11. 22.  5.  8. 21.  1. 10. 20. 17.  7.  2.  6. 15. 24. 14. 19.
 16.  0. 13. 12. 18.  9.  3.]
1074.9129252846774
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24.]
True
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 